In [ ]:
import re
from pathlib import Path
from collections import Counter
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup   

dataset preparation

In [ ]:
def epub_to_txt(epub_path: str, txt_path: str) -> None:
    """
    Extract plain text from an EPUB file, run the same post-processing
    pipeline, and save as UTF-8 .txt.

    Parameters
    ----------
    epub_path : str | Path
        Input .epub file.
    txt_path  : str | Path
        Target .txt file.
    """
    # read the entire EPUB package
    book = epub.read_epub(epub_path)

    # concatenate all xhtml/html documents in spine order
    chunks = []
    for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
        soup = BeautifulSoup(item.get_content(), "lxml")
        chunks.append(soup.get_text(separator="\n"))

    raw_text = "\n\n".join(chunks)

    # reuse your existing postprocess() function
    cleaned = postprocess(raw_text)

    # save
    Path(txt_path).write_text(cleaned, encoding="utf-8")
    print(f"Saved => {txt_path}")


Epub post processing

In [ ]:

def postprocess(raw: str) -> str:
    """
    对 PDF/EPUB 提取出的原始文本进行清洗，返回 UTF-8 字符串。
    步骤：
    0) 换行/控制字符标准化
    1) 去页码 + 高频页眉页脚
    2) 合并断行，保留空行作段落分隔
    3) 修复跨行连字符 “hy-\nphen”
    4) 折叠多余空行
    5) 可选：标点半/全角转换、繁简体转换等
    """

    # ------------------------------------------------------------------ #
    # 0) 统一换行符 & 去除控制字符                                        #
    # ------------------------------------------------------------------ #
    txt = (
        raw.replace("\r\n", "\n")          # CRLF → LF
           .replace("\r", "\n")            # CR → LF
           .lstrip("\ufeff")               # 去 UTF-8 BOM
    )
    # 删除除 LF/TAB 外的其他 ASCII 控制字符
    txt = re.sub(r"[\x00-\x09\x0B-\x1F\x7F]", "", txt)

    lines = txt.split("\n")

    # ------------------------------------------------------------------ #
    # 1) 去孤立页码 & 高频页眉/页脚                                      #
    # ------------------------------------------------------------------ #
    pat_page = re.compile(r"^\s*(?:Page\s*)?\d{1,4}\s*(?:页|Page)?\s*$")
    trimmed = [ln.strip() for ln in lines if ln.strip()]
    # 统计出现频次，出现率超过阈值且长度 <80 的行视为模板化噪声
    common  = {ln for ln, c in Counter(trimmed).items()
                     if c > len(lines) * _HEADER_THRESHOLD and len(ln) < 80}

    def is_noise(line: str) -> bool:
        return pat_page.match(line) or line.strip() in common

    content_lines = [ln for ln in lines if not is_noise(ln)]

    # ------------------------------------------------------------------ #
    # 2) 合并断行：非句末行与下一行拼接                                  #
    # ------------------------------------------------------------------ #
    merged, buf = [], ""
    for ln in content_lines:
        if not ln.strip():           # 空行 → 段落边界
            if buf:
                merged.append(buf)
                buf = ""
            continue

        buf += ln.strip()            # 去两端空白再拼
        if re.search(_SENT_END + r"$", ln):  # 句末标点→刷新缓冲
            merged.append(buf)
            buf = ""
        else:
            buf += " "               # 行内继续，加空格

    if buf:
        merged.append(buf)

    # ------------------------------------------------------------------ #
    # 3) 修复 “hy-\nphen” 这类跨行连字符                                 #
    # ------------------------------------------------------------------ #
    text_block = "\n".join(merged)
    text_block = re.sub(r"-\s*\n([a-zA-Z])", r"\1", text_block)

    # ------------------------------------------------------------------ #
    # 4) 连续 ≥3 个换行压缩成 1 个空行                                   #
    # ------------------------------------------------------------------ #
    text_block = re.sub(r"\n{3,}", "\n\n", text_block)

    # ------------------------------------------------------------------ #
    # 5) 可选增强：如需可取消注释                                        #
    # ------------------------------------------------------------------ #
    # # a) 全角书名号《》 → 半角引号
    # text_block = text_block.replace("《", "\"").replace("》", "\"")
    #
    # # b) 繁体转简体
    # # from opencc import OpenCC
    # # text_block = OpenCC("t2s").convert(text_block)

    return text_block.strip()

In [ ]:
from pathlib import Path
epub_to_txt("北京枪声 (Unknown) (Z-Library).epub", "北京枪声 (Unknown) (Z-Library).epub.txt")
txt_path = Path("北京枪声 (Unknown) (Z-Library).epub.txt")
print(txt_path.exists())          # True → 文件确实写好了
print(txt_path.stat().st_size)  

In [ ]:
from bertalign.aligner import Aligner
import nltk, regex
nltk.download("punkt")
aligner = Aligner(model_name="sentence-transformers/LaBSE")  # 会自动下载模型

def split_en(text):                 # NLTK 英文分句
    return nltk.sent_tokenize(text)

def split_zh(text):                 # 正则中文分句
    return [s for s in regex.split(r'(?<=[。！？])\s*', text) if s.strip()]

def align_sentences(en_sents, zh_sents):
    """
    返回按顺序对应的 [(en_idx, zh_idx)] 列表；Bertalign 内部已做动态规划
    """
    pairs = aligner.align(en_sents, zh_sents, threshold=0.5)
    return pairs